Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist im Beispiel ein Apache-Webserver installiert, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl label  namespace ms-rest istio-injection=enabled
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml
kubectl get service/apache --namespace ms-rest

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

Wird dieser URL in einem neuen Tab geöffnet (rechte Maustaste), wird ein Menu angezeigt.

In [ ]:
! [ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest apache -o=jsonpath='{ .spec.ports[0].nodePort }')

---
## Kiali — Observability


Kiali vom, Projekt Istio, hilft die Struktur des Servicenetzes und deren Topologie zu verstehen.

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

- - - 
## Jaeger — Tracing

Jaeger ist ein System, um Aufrufe zwischen Microservices zu verfolgen..

Jaeger ist jedoch nur auf dem Cluster zugänglich. Deshalb müssen wir Jaeger zuerst freischalten bzw. gegen NodePort öffnen:

In [ ]:
! kubectl get service -n istio-system -l app=jaeger -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Jaeger  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/tracing -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Grafana

Grafana ist eine plattformübergreifende Open-Source-Anwendung zur grafischen Darstellung von Daten aus verschiedenen Datenquellen wie z. B. InfluxDB, MySQL, PostgreSQL, Prometheus und Graphite. Die erfassten Rohdaten lassen sich anschließend in verschiedenen Anzeigeformen ausgeben

In [ ]:
! kubectl get service -n istio-system -l app.kubernetes.io/instance=grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Grafana  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app.kubernetes.io/instance=grafana -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')                                                         

- - -
## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

In [ ]:
%%bash
for i in `seq 1 1000`;
do
    curl -s -o /dev/null -I -w "%{http_code}" http://apache.ms-rest/order/order
    echo
done

- - - 
### REST Paradigma

Representational State Transfer (abgekürzt REST, seltener auch ReST) bezeichnet ein Programmierparadigma für verteilte Systeme.

REST ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web (HTTP GET, PUT, POST, DELETE).

REST hat das Ziel, einen Architekturstil zu schaffen, der die Anforderungen des modernen Web besser darstellt. Dabei unterscheidet sich REST vor allem in der Forderung nach einer einheitlichen Kommunikation Schnittstelle von anderen Architekturstilen.

Um das REST Paradigma zu verdeutlichen, führen wir mittels `curl` (Windows `Invoke-WebRequest`) mehrere API Aufrufe gegen unsere Microservices aus:

In [ ]:
! curl http://apache.ms-rest/customer/customer

In [ ]:
! curl http://apache.ms-rest/catalog/catalog

In [ ]:
! curl http://apache.ms-rest/order/order/<Ersetzen mit order id>

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-rest